# 12 비정형_Audio
# 1. 분석 환경 준비
## 1.1. 데이터 불러오기
분석하려는 데이터를 가져오는 작업

* 파이썬 라이브러리 Pandas 이용

* glob() 메소드 이용하여 특정한 패턴이나 확장자를 가진 파일들의 경로나 이름을 리스트 형식으로 반환

# 2. 데이터 전처리
## 2.1. WAV 파일 전처리

* librosa.load를 통해 wav 파일의 signal과 sample rate(일정한 시간 간격) 추출

## 2.2. 음성 길이 설정

음성 중 길이가 가장 작은 길이의 데이터를 기준으로 데이터를 잘라서 같은 길이로 설정

* pad2d() 메소드 사용
* 혹은 직접 음성 데이터 자르기

## 2.3. 음성 데이터 특징 추출

입력된 신호에서 노이즈 및 배경 소리로 부터 실제로 유용한 소리의 특징을 추출

### 2.3.1. Mel Spectrogram   
낮은 주파수를 높은 주파수보다 더 예민하게 받아들이므로 이 주파수를 mel scale로 볼 수 있게 하는 음성의 특징 추출 방법

* 음성 데이터를 raw data를 그대로 사용하면 파라미터가 너무 많아지기도 하고 데이터 용량이 너무 커지므로 mel spectrogram을 사용
* mel_spectrogram() 메소드 이용
    * 1. n_fft : 한 번 fft(Fast Fouriter Transform)를 해 줄 만큼의 sequence 길이 (음성의 길이를 얼마만큼으로 자를 것인가 = window)
    * 2. hop_length : window 간의 거리 

### 2.3.2. MFCC (Mel-frequency cepstral coefficients)

입력된 소리 전체를 대상으로 하는 것이 아니라, 일정 구간(Short time)식 나누어 이 구간에 대한 스펙트럼을 분석하여 특징 추출

* librosa.feature.mfcc(wav) 메소드 이용
    *  sr : sampling rate
    * n_mfcc :  return 될 mfcc의 개수를 정해주는 파라미터. 더 다양한 데이터 특징을 추출하기 위해서 증가 시킴
    * n_fft : frame의 length를 결정하는 파라미터. n_fft를 설정하면 window size가 자동으로 같은 값으로 설정. n_fft는 window size보다 크거나 같아야 함   
        일반적으로 자연어 처리에서는 음성을 25m의 크기를 기본으로 하고 있음   
        (ex. 16000Hz인 음성에서는 400에 해당하는 값 (16000 * 0.025 = 400) 즉, n_fft는 sr에 frame_length인 0.025를 곱한 값)
    * hop_length : 길이만큼 옆으로 가면서 데이터를 읽습니다.

## 2.4. 스케일링

음성 데이터에서 추출한 특징을 전처리 scaling

* 직접 : (mel - mel.mean()) / mel.std() 
* 메소드 사용 : sklearn.preprocessing.scale(mfcc, axis=1)
 

## 2.5. 시각화

특징 추출된 음성 데이터의 시각적인 특징을 보기 위하여 시각화

*  librosa 라이브러리의 specshow() 사용

## 2.6. 데이터 차원 확대 

딥러닝 모델에 넣기 위하여 데이터 차원 확대

* 방법 1 : np.expand_dims(train_mfccs, -1)
* 방법 2 : train_x.reshape(-1, train_x.shape[1], train_x.shape[2], 1)

# 3. 모델링

## 3.1. 변수 정의

X : 독립 변수 (음성 데이터)   
y : 종속 변수 (라벨 ,타겟)

## 3.2. CNN
Convolution이라는 전처리 작업이 들어가는 Neural Network 모델

* 음성 데이터에서도 CNN 사용


### 3.2.1. CNN 모델 설계

keras.Sequential

* Conv2D :  이미지의 특징(feature map)을 추출해내는 역할
    
    * Stride : 커널을 이동시키는 거리
    * Padding : 이미지 주위에 0을 둘러서 이미지 데이터의 축소를 방지
    * activation : 활성화 함수 설정
    
* MaxPooling2D : 데이터의 공간적 크기를 축소하는데 사용
* Dropout : 뉴럴 네트워크의 유닛(뉴런)들을 랜덤으로 비활성화 하여 과적합(Overfittng)을 방지
* Flatten : N차원 배열을 1차원으로 변환
* Dense : 1차원 배열을 뉴럴넷에 입력 (오로지 퍼셉트론으로 이루어짐)

### 3.2.2. 모델 컴파일

model.complie   
생성 모델과 학습 알고리즘(옵티마이저)을 붙이기 위하여 학습할 수 있는 상태로 만들어 주도록 컴파일

* optimizer : 최적화 방법 
* loss : 손실함수, 회귀의 경우 mse, 분류일 경우 categorical_crossentropy
* metrix : 평가지표

### 3.2.3. 모델 학습

history = model.fit

* 모델을 history 라는 매개변수에 저장하게 되면 모델에서 학습했던 자취들이 남아있도록 함
* epochs : 에포크
* validation_split : 모델 검증
* batch_size : 배치사이즈
* callbacks : 콜백함수 사용
    * EarlyStopping : 모델 성능 향상이 없는 경우 학습을 중지
    * ModelCheckpoint : 가장 validation performance 가 좋은 모델을 저장




# 4. 모델 예측

## 4.1. 분류 모델 지표
### 4.1.1. Accuracy
올바르게 예측된 데이터의 수를 전체 데이터의 수로 나눈 값

### 4.1.2. F1-score
정밀도와 재현율의 조화 평균

precision과 recall이 0 에 가까울수록 F1 score도 동일하게 낮은 값